In [23]:
import pandas as pd
import toml
from pathlib import Path

config = toml.load(Path.cwd() / '../../../../configuration/input_configuration.toml')
summary_config = toml.load(Path.cwd() / '../../../../configuration/summary_configuration.toml')

pd.set_option('display.float_format', '{:,.0%}'.format)


In [3]:
# Relative path between notebooks and goruped output directories
output_path = Path(summary_config['sc_run_path']) / summary_config["output_folder"]
survey_path = Path(summary_config['sc_run_path']) / summary_config["survey_folder"]

In [7]:
# trip data
hh = pd.read_csv(output_path / 'agg/dash/auto_ownership_efa.csv')

# list of equity geographies
equity_geogs = summary_config['hh_equity_geogs']
# not_equity_geogs = ["NOT in " + item for item in equity_geogs]

In [ ]:
# TRIPS
df_hh = hh.copy()
# add home RGC
df_hh['is_rgc'] = 'Not in RGC'
df_hh.loc[df_hh['hh_rgc'] != 'Not in RGC', 'is_rgc'] = 'In RGC'

equity_geogs = ['hh_efa_dis', 'hh_efa_older', 'hh_efa_lep', 'hh_efa_pov200', 'hh_efa_poc', 'hh_efa_youth']
df_hh[equity_geogs] = df_hh[equity_geogs].apply(lambda x: x.\
        map({0: 'Below Regional Average', 
             1: 'Above Regional Average', 
             2: 'Higher Share of Equity Population'}))


## Vehicle Ownership by Equity Focus Areas

In [26]:
def stat_by_equity_geog(df, geog, group):
    """
    Function to calculate statistics by equity geography and vehicle ownership.
    """
    # Group by equity geography and vehicle ownership
    df_grouped = df.groupby([geog, group], as_index=False)['hhexpfac'].sum()
    
    # Calculate total households in each equity geography
    total_hh = df.groupby([geog], as_index=False)['hhexpfac'].sum().rename(columns={'hhexpfac': 'total_hh'})
    
    # Merge the grouped data with total households
    df_merged = df_grouped.merge(total_hh, on=geog)
    
    # Calculate percentage of households with the specified vehicle ownership
    df_merged['percentage'] = df_merged['hhexpfac'] / df_merged['total_hh']
    
    return df_merged.pivot(index=geog, columns=group, values='percentage')


In [27]:
stat_by_equity_geog(df_hh, 'hh_efa_poc', 'hhvehs')

hhvehs,0,1,2,3,4
hh_efa_poc,,,,,
Above Regional Average,10%,34%,35%,14%,8%
Below Regional Average,6%,31%,38%,17%,9%
Higher Share of Equity Population,10%,37%,34%,13%,7%


In [28]:
stat_by_equity_geog(df_hh, 'hh_efa_pov200', 'hhvehs')

hhvehs,0,1,2,3,4
hh_efa_pov200,,,,,
Above Regional Average,8%,34%,35%,14%,8%
Below Regional Average,6%,30%,39%,17%,9%
Higher Share of Equity Population,13%,39%,31%,11%,6%


In [29]:
stat_by_equity_geog(df_hh, 'hh_efa_lep', 'hhvehs')

hhvehs,0,1,2,3,4
hh_efa_lep,,,,,
Above Regional Average,8%,32%,37%,15%,8%
Below Regional Average,8%,32%,37%,16%,8%
Higher Share of Equity Population,7%,36%,35%,13%,8%


In [31]:
stat_by_equity_geog(df_hh, 'hh_efa_dis', 'hhvehs')

hhvehs,0,1,2,3,4
hh_efa_dis,,,,,
Above Regional Average,8%,34%,35%,15%,8%
Below Regional Average,6%,31%,38%,16%,8%
Higher Share of Equity Population,12%,35%,32%,13%,8%


In [32]:
stat_by_equity_geog(df_hh, 'hh_efa_older', 'hhvehs')

hhvehs,0,1,2,3,4
hh_efa_older,,,,,
Above Regional Average,7%,31%,37%,17%,9%
Below Regional Average,9%,34%,36%,14%,7%
Higher Share of Equity Population,7%,31%,36%,17%,9%


In [33]:
stat_by_equity_geog(df_hh, 'hh_efa_youth', 'hhvehs')

hhvehs,0,1,2,3,4
hh_efa_youth,,,,,
Above Regional Average,4%,28%,40%,18%,10%
Below Regional Average,11%,38%,33%,12%,7%
Higher Share of Equity Population,3%,24%,43%,20%,10%
